In [ ]:
import os
from ultralytics import YOLO
from PIL import Image
from pathlib import Path
from typing import List, Dict
from tqdm.auto import tqdm
import json
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
from sane_coco.dataset import COCODataset
from sane_coco.metrics import MeanAveragePrecision


%load_ext autoreload
%autoreload 2

def ultralytics_batch_detect(
    image_paths: List[str], 
    batch_size: int = 16, 
    conf: float = 0.25
) -> List[List[Dict]]:
    model = YOLO('yolov8n.pt', verbose=False)
    model.to('cpu')
    results = []
    
    for i in tqdm(range(0, len(image_paths), batch_size)):
        batch = image_paths[i:i + batch_size]
        preds = model(batch, conf=conf, verbose=False)
        batch_results = [
            [
                {'bbox': box.xyxy[0].tolist(), 
                 'conf': float(box.conf), 
                 'class': int(box.cls)}
                for box in pred.boxes
            ]
            for pred in preds
        ]
        results.extend(batch_results)
    return results


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [176]:
YOLO_TO_COCO = {
    0: 1, 1: 2, 2: 3, 3: 4, 4: 5, 5: 6, 6: 7, 7: 8, 8: 9, 9: 10, 10: 11, 11: 13, 12: 14, 
    13: 15, 14: 16, 15: 17, 16: 18, 17: 19, 18: 20, 19: 21, 20: 22, 21: 23, 22: 24, 23: 25,
    24: 27, 25: 28, 26: 31, 27: 32, 28: 33, 29: 34, 30: 35, 31: 36, 32: 37, 33: 38, 34: 39,
    35: 40, 36: 41, 37: 42, 38: 43, 39: 44, 40: 46, 41: 47, 42: 48, 43: 49, 44: 50, 45: 51,
    46: 52, 47: 53, 48: 54, 49: 55, 50: 56, 51: 57, 52: 58, 53: 59, 54: 60, 55: 61, 56: 62,
    57: 63, 58: 64, 59: 65, 60: 67, 61: 70, 62: 72, 63: 73, 64: 74, 65: 75, 66: 76, 67: 77,
    68: 78, 69: 79, 70: 80, 71: 81, 72: 82, 73: 84, 74: 85, 75: 86, 76: 87, 77: 88, 78: 89,
    79: 90
}


In [177]:
images_dir = 'COCO/DIR/val2017'
image_filenames = os.listdir(images_dir)
image_paths = [f'{images_dir}/{image}' for image in image_filenames]
results = ultralytics_batch_detect(image_paths, batch_size=10)

100%|██████████| 500/500 [04:27<00:00,  1.87it/s]


# Eval using pycocotools

In [183]:
annotations_fpath = '/Users/boris/Documents/datasets/coco/annotations/instances_val2017.json'

predictions_fpath = './predictions.json'

In [179]:
anno = COCO(str(annotations_fpath)) 

predictions = []
for i, (image_filename, result) in enumerate(zip(image_filenames, results)):
    image_id = int(Path(image_filename).stem)
    for detection in result:
        predictions.append({
            'image_id': image_id,
            'bbox': detection['bbox'],
            'score': detection['conf'],
            'category_id': YOLO_TO_COCO[detection['class']]
        })

with open(predictions_fpath, 'w') as f:
    json.dump(predictions, f)

pred = anno.loadRes(str(predictions_fpath))

val = COCOeval(anno, pred, "bbox")
val.params.imgIds = [int(Path(x).stem) for x in image_filenames]
val.evaluate()
val.accumulate()
val.summarize()
stats = {}
stats["map_50_95"] = val.stats[0]
stats["map_50"] = val.stats[1]

print('MAP@0.5|0.95:', stats['map_50_95'])
print('MAP@0.5:', stats['map_50'])

loading annotations into memory...
Done (t=0.21s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.04s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=4.11s).
Accumulating evaluation results...
DONE (t=0.57s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.010
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.031
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.006
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.005
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.020
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.031
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.036
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets

# Eval using sane coco

In [180]:
with open(annotations_fpath, 'r') as f:
    annotations = json.load(f)

dataset = COCODataset.from_dict(annotations)
annotations = dataset.get_annotation_dicts()

included_images = [
    i for i in range(len(dataset.images))
    if dataset.images[i].id in [int(Path(x).stem) for x in image_filenames]
]
included_image_ids = [dataset.images[i].id for i in included_images]
included_annotations = [annotations[i] for i in included_images]

In [181]:
predicted_image_ids = [int(Path(x).stem) for x in image_filenames]
predictions = {}
for i, (image_id, result) in enumerate(zip(predicted_image_ids, results)):
    image_predictions = []
    for detection in result:
        category_id = YOLO_TO_COCO[detection['class']]
        category = dataset.get_category_by_id(category_id)

        image_predictions.append({
            'score': detection['conf'],
            'category': category.name,
            'bbox': detection['bbox'],
        })
    predictions[image_id] = image_predictions

predictions = [predictions[i] for i in included_image_ids]

In [182]:
metrics = MeanAveragePrecision()
metrics.update(included_annotations, predictions)
stats = metrics.compute()
print('MAP@0.5|0.95:', stats['map'])
print('MAP@0.5:', stats['ap'][0.5])

MAP@0.5|0.95: 0.008281212656477256
MAP@0.5: 0.024394058646580518
